In [23]:
import pandas as pd
import xarray as xr
import numpy as np
from numpy import errstate,isneginf,array
import datetime
import os
import yaml

import matplotlib.pyplot as plt
import cmocean as cm    

from xgboost import XGBRegressor

In [24]:
#This file contains configuration details like API keys and passwords
global_vars = yaml.safe_load(open('../config.yml', 'r') )

In [25]:
#This has custom functions - log transform
%run ./00_custom_functions.ipynb

In [26]:
#Set base folders
cloud = True
if cloud:
    result_folder = global_vars['reconstruction_folder_cloud']
    data_folder_root = global_vars['download_folder_cloud']
else:
    result_folder = global_vars['reconstruction_folder_local']
    data_folder_root = global_vars['download_folder_local']
print(result_folder)

/data/artemis/workspace/ds4114/reconstructions/


In [37]:
#Set location of input files (path from root above)
#Note that these were processed such that they already share a coordinate set
sst_processed  = data_folder_root + 'SST/processed/SST_NOAA_OI-V2-1x1_198201-202304.nc'
sss_processed  = data_folder_root + 'SSS/processed/SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_198201-202304.nc' #previously 202303
mld_processed  = data_folder_root + 'MLD/processed/MLD_IFREMER-deBoyer_DT02-c1m-1x1_198201-202304.nc'
chl_processed  = data_folder_root + 'CHL/processed/CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_198201-202304.nc'
pco2_processed = data_folder_root + 'pCO2/processed/pCO2_LEAP_SOCAT-ERA5-weighted_198201-202212.nc'
xco2_processed = data_folder_root + 'xCO2/processed/xCO2_NOAA_xCO2-mm-gl-monthly_198201-202303.nc'
#add additional sources if desired
sst_processed_option2 = data_folder_root + 'SST/processed/SST_ECMWF_ERA5-monthly-reanalysis-1x1-SST_198201-202304.nc'
sst_processed_option3 = data_folder_root + 'SST/processed/SST_JMA_JRA55-do-monthly-reanalysis-SST_198201-202304.nc'
mld_processed_option2 = data_folder_root + 'MLD/processed/MLD_UCSD-Argo_MLD-dt-mean-1x1_198201-202304.nc'
list_for_df = [sst_processed, sss_processed, mld_processed, chl_processed, pco2_processed, xco2_processed
              ,sst_processed_option2, sst_processed_option3, mld_processed_option2
              ] 

In [42]:
#This is where we set parameters for the ML algorithm for finding the long term pco2 mean feature

#The next variable is for the XGBoost method for both pCO2 Residual and creating the long term pCo2 mean feature. They were determined via a grid search in previous iterations. 
best_params = {'max_depth': 9, 'n_estimators': 1000} 
random_seed = 47  #Set the random seeds used for training
jobs = -1         #Number of cores you have access to for model training; -1 for all available ones

#This variable is a list of features used for the Long Term pCO2 mean machine learning
feature_sel = ['sst','sst_anomaly','sss','sss_anomaly','chl_log','chl_log_anomaly','mld_log','xco2_trend','A','B','C','T0','T1']
target_sel = ['fco2']  #previously was pco2
sst_variable_option = ['sst']  #the name of the SST variable to use for calculating the residual component of the target

# Create Features
### Base Features

In [38]:
xrfull = xr.merge([xr.open_dataset(f) for f in list_for_df], compat='broadcast_equals')
xrfull.attrs = "" #just removing attribute details since wont be accurate anymore
#xrfull

<xarray.Dataset>
Dimensions:     (time: 496, xlon: 360, ylat: 180)
Coordinates:
  * time        (time) datetime64[ns] 1982-01-15 1982-02-15 ... 2023-04-15
  * xlon        (xlon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * ylat        (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    sst         (time, ylat, xlon) float32 ...
    sss         (time, ylat, xlon) float32 ...
    mld         (time, ylat, xlon) float64 ...
    chl         (time, ylat, xlon) float64 ...
    pco2        (time, ylat, xlon) float64 nan nan nan nan ... nan nan nan nan
    socat_sst   (time, ylat, xlon) float32 nan nan nan nan ... nan nan nan nan
    mslp        (time, ylat, xlon) float32 1.006e+03 1.006e+03 ... nan nan
    fco2        (time, ylat, xlon) float32 nan nan nan nan ... nan nan nan nan
    xco2_trend  (time) float64 340.7 340.9 340.8 340.9 ... 418.3 418.3 418.5 nan
    sst_era5    (time, ylat, xlon) float32 ...
    sst_jra55   (time, ylat, xlon) float32 ...
    mld_argo    (time, ylat, xlon) float64 ...

In [39]:
#next add derived (logs)
xrfull = xrfull.assign( mld_log = log_or_0_xr(xrfull.mld, 'mld_log') 
                       ,chl_log = log_or_0_xr(xrfull.chl, 'chl_log')
                       ,mld_argo_log = log_or_0_xr(xrfull.mld_argo, 'mld_argo_log')
                      )

In [40]:
%%time
#add anomalies fields
anomalies = xrfull.groupby("time.month") - xrfull.groupby("time.month").mean("time")
anomalies = anomalies.get(['sst','sss','chl_log', 'sst_era5', 'sst_jra55']) #just need SST, SSS, CHL anomalies
anomalies = anomalies.drop('month') 
anomalies = anomalies.rename({ 'sst': 'sst_anomaly'
                              ,'sss':'sss_anomaly'
                              ,'chl_log':'chl_log_anomaly'
                              ,'sst_era5':'sst_era5_anomaly'
                              ,'sst_jra55':'sst_jra55_anomaly'
                             }) 
#anomalies
xrfull = xrfull.merge(anomalies, compat='identical')

CPU times: user 7.74 s, sys: 7.11 s, total: 14.9 s
Wall time: 19.5 s


In [41]:
#add time and space derivations
xrfull = xrfull.assign( days_idx = xrfull.time.dt.dayofyear 
                       ,lon_rad = np.radians(xrfull.xlon)
                       ,lat_rad = np.radians(xrfull.ylat)
                      )
xrfull = xrfull.assign( T0 = np.cos(xrfull.days_idx * 2 * np.pi / 365)
                       ,T1 = np.sin(xrfull.days_idx * 2 * np.pi / 365)
                       ,A  = np.sin(xrfull.lat_rad)
                       ,B  = np.cos(xrfull.lat_rad)*np.sin(xrfull.lon_rad)
                       ,C  = -np.cos(xrfull.lat_rad)*np.cos(xrfull.lon_rad)
                      )

### Set Data Fields/Points For Learning Long Term Mean

In [43]:
%%time
xr_for_ocean_co2_mean = xrfull.get(feature_sel + target_sel)  #Note - we train on all available data (no time slice is used here)
df_for_ocean_co2_mean = xr_for_ocean_co2_mean.to_dataframe() #expensive function; need lots of RAM 
df_for_ocean_co2_mean_to_train = df_for_ocean_co2_mean[(~df_for_ocean_co2_mean.isna().any(axis=1))]  #only keep points that are not null 
print(f'Number of points in time/space for training: {df_for_ocean_co2_mean_to_train.shape[0]}')
#df_for_ocean_co2_mean_to_train

df_for_ocean_co2_mean_to_predict_temp = df_for_ocean_co2_mean.loc[:,feature_sel]
df_for_ocean_co2_mean_to_predict = df_for_ocean_co2_mean_to_predict_temp[(~df_for_ocean_co2_mean_to_predict_temp.isna().any(axis=1))]  #only predict on points with all variables globally
print(f'Number of points in time/space available to reconstruct: {df_for_ocean_co2_mean_to_predict.shape[0]}')
#df_for_ocean_co2_mean_to_predict

Number of points in time/space for training: 313433
Number of points in time/space available to reconstruct: 18309940
CPU times: user 6.66 s, sys: 9.3 s, total: 16 s
Wall time: 16 s


### ML for Ocean CO2_Long_Term_Mean Feature

In [ ]:
%%time
X_train = df_for_ocean_co2_mean_to_train.loc[:,feature_sel]
y_train = df_for_ocean_co2_mean_to_train.loc[:,target_sel]

model = XGBRegressor(random_state=random_seed, **best_params, n_jobs=jobs)
print(f'Training started on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'...')
model.fit(X_train, y_train)    #training on all data with no cross validation because we are only calculating a long term average
                               #Model evalation for pco2 residual is in the next script
print('Training Complete on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'. Predicting...')
ocean_co2_for_mean_recon = model.predict(df_for_ocean_co2_mean_to_predict)
print("Complete")  #Training and predicting may take 15 minutes

In [ ]:
#average across time and add back to dataset 
ocean_co2_for_mean_recon_xr = pd.DataFrame(ocean_co2_for_mean_recon,index=df_for_ocean_co2_mean_to_predict.index,columns=['ocean_co2_recon_for_mean']).to_xarray()
xrfull = xrfull.merge(ocean_co2_for_mean_recon_xr, compat='identical') #add back to full set
xrfull = xrfull.assign(ocean_co2_mean = xrfull.ocean_co2_recon_for_mean.mean('time'))

In [14]:
#Export if desired. Some extra code to find the dates used
if True:
    min_yearmonth = str(ocean_co2_for_mean_recon_xr.time.min().data.astype('datetime64[s]').item().strftime('%Y%m')) #just gets the min date from the xarray in YYYYMM format
    max_yearmonth = str(ocean_co2_for_mean_recon_xr.time.max().data.astype('datetime64[s]').item().strftime('%Y%m')) 
    ocean_co2_long_term_mean = ocean_co2_for_mean_recon_xr.mean('time')
    ocean_co2_long_term_mean = ocean_co2_long_term_mean.rename({'ocean_co2_recon_for_mean': str(target_sel[0])+'_mean'})
    ocean_co2_output_name = result_folder+'pCO2_LEAP_XGBoost-'+str(target_sel[0])+'-long-term-mean-from-'+min_yearmonth+'-to-'+max_yearmonth+'.nc'
    ocean_co2_long_term_mean.to_netcdf(ocean_co2_output_name)
    print(f'Saved: {ocean_co2_output_name}')

Saved: /data/artemis/workspace/ds4114/reconstructions/pCO2_LEAP_XGBoost-fco2-long-term-mean-from-198201-to-202303.nc


### Residual (pCO2 T and NonT Features)

In [55]:
xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0413 * (xrfull[sst_variable_option[0]] - xrfull[sst_variable_option[0]].mean("time"))) ) # Wanninkhof et al. 2022
    #previously: xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0423* (xrfull.sst - xrfull.sst.mean("time"))) ) # Takahashia et al, 2004
xrfull = xrfull.assign(ocean_co2_nonT = xrfull[target_sel[0]] - xrfull.ocean_co2_T)

In [ ]:
#Perform some clean up before exporting

xrfull = xrfull.drop_vars(['ocean_co2_recon_for_mean'])  #optionally could keep/drop this variable
xrfull = xrfull.rename({'ocean_co2_T': str(target_sel[0])+'_T'
                        ,'ocean_co2_nonT': str(target_sel[0])+'_nonT'
                        ,'ocean_co2_mean': str(target_sel[0])+'_mean'
                        #,'ocean_co2_recon_for_mean': str(target_sel[0])+'_recon_for_mean'
                       })

#add attributes as needed here
xrfull.attrs['description'] = "fCO2-Residual dataset for machine learning"
xrfull.attrs['formula_for_co2_residual'] = 'uses 0.0413 factor from Wanninkhof et al. 2022 and temperature variable: '+ str(sst_variable_option[0])
xrfull.attrs['training_features_for_co2_mean'] = str(feature_sel)
xrfull.attrs['hyperparameters_for_co2_mean'] = str(best_params)
xrfull.attrs['created'] = str(datetime.datetime.now())
#xrfull.attrs['methodology'] = "Bennington et al. (2022), JAMES"

In [58]:
xrfull

<xarray.Dataset>
Dimensions:            (time: 496, xlon: 360, ylat: 180)
Coordinates:
  * ylat               (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * time               (time) datetime64[ns] 1982-01-15 ... 2023-04-15
  * xlon               (xlon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Data variables: (12/31)
    sst                (time, ylat, xlon) float32 nan nan nan ... -1.74 -1.719
    sss                (time, ylat, xlon) float32 nan nan nan ... 31.74 31.76
    mld                (time, ylat, xlon) float64 0.0 0.0 0.0 ... 103.6 104.2
    chl                (time, ylat, xlon) float64 nan nan nan ... nan nan nan
    pco2               (time, ylat, xlon) float64 nan nan nan ... nan nan nan
    socat_sst          (time, ylat, xlon) float32 nan nan nan ... nan nan nan
    ...                 ...
    A                  (ylat) float64 -1.0 -0.9997 -0.999 ... 0.999 0.9997 1.0
    B                  (ylat, xlon) float64 -7.615e-05 -0.0002284 ... 7.615e-05
    C                  (ylat, xlon) float64 0.008726 0.008724 ... 0.008726
    fco2_mean          (ylat, xlon) float32 nan nan nan nan ... nan nan nan nan
    fco2_T             (time, ylat, xlon) float32 nan nan nan ... nan nan nan
    fco2_nonT          (time, ylat, xlon) float32 nan nan nan ... nan nan nan
Attributes:
    description:                     fCO2-Residual dataset for machine learning
    formula_for_co2_residual:        uses 0.0413 factor from Wanninkhof et al...
    training_features_for_co2_mean:  ['sst', 'sst_anomaly', 'sss', 'sss_anoma...
    hyperparameters_for_co2_mean:    {'max_depth': 9, 'n_estimators': 1000}
    created:                         2023-07-06 17:03:40.824354

In [59]:
#export out (may be 3GB unless some variables dropped)
output_netcdf_with_date(xrfull, result_folder+'', 'pCO2_LEAP_'+str(target_sel[0])+'-residual-full-dataset-preML')

Saved pCO2_LEAP_fco2-residual-full-dataset-preML_198201-202304.nc to /data/artemis/workspace/ds4114/reconstructions/


## For Debugging